In [ ]:
# Wiring:
#  BNO055 orientation accelerometer to ESP32: Vin=PS1=3V3, GND=GND, SDA=RX2, SCL=TX2
#  PX4AIRSPEEDV1.2i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)


In [79]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [10]:
%sendtofile config.txt

pinled       2
deviceletter @
connection0  JulianS9,beckaaaa,192.168.43.1,9042
dev1         1,rx16,tx17,Z

Sent 4 lines (105 bytes) to config.txt.


In [7]:
%sendtofile --source BNO055_class.py
%sendtofile --source UDPblackviewphone.py


Sent 42 lines (1123 bytes) to UDPblackviewphone.py.


In [84]:
from machine import I2C, Pin
import time

i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)

i2cdevices = i2c.scan()
if 0x28 in i2cdevices:
    xbs = bytearray("Xt00000000d0000r0000\n")
    mxbs = memoryview(xbs)
    tstamppx4 = 0
    print("PX4 pitot found")
else:
    xbs = None
    print("PX4 pitot not found")
px4readrate = 400
    
def readi2cdevices(tstamp):
    global tstamppx4
    if xbs != None:
        if tstamppx4 == 0:
            tstamppx4 = tstamp + px4readrate
            x = i2c.readfrom(0x28, 4)
            #status = x[0] & 0xC0  # should be zero
            rawpress = ((x[0] & 0xFF) << 8) | (x[1])
            #rawpress = ((x[0] & 0x3F) << 8) | (x[1])
            rawtemp = (x[2] << 3) | (x[3] >> 5)
            mxbs[2:10] = b"%08X" % tstamp
            mxbs[11:15] = b"%04X" % rawpress
            mxbs[16:20] = b"%04X" % rawtemp
            return xbs
            #psi = rawpress*(1.0/(0x3FFF*0.4)) - 1.25
            #diffpresspascals = psi*6894.75728
            #temp = rawtemp*200.0/0x7FF - 50
            #airdensity = 1.03
            #airspeed = 2*diffpresspascals/airdensity

        elif tstamp > tstamppx4:
            tstamppx4 = 0
            i2c.writeto(0x28, b"")
    return None


PX4 pitot found


In [85]:
while True:
    tstamp = time.ticks_ms()
    v = readi2cdevices(tstamp)
    if v is not None:
        print(v)


bytearray(b'Xt0001244Ed2053r02C3\n')
bytearray(b'Xt000125DFd2055r02C3\n')
bytearray(b'Xt00012770d2058r02C3\n')
bytearray(b'Xt00012901dA056r02C3\n')
bytearray(b'Xt00012A92dA059r02C3\n')
bytearray(b'Xt00012C23d2059r02C3\n')
bytearray(b'Xt00012DB4d2059r02C3\n')
bytearray(b'Xt00012F45dA059r02C3\n')
bytearray(b'Xt000130D6d2059r02C3\n')
bytearray(b'Xt00013267dA059r02C3\n')
bytearray(b'Xt000133F8d2056r02C3\n')
bytearray(b'Xt00013589d2055r02C3\n')
bytearray(b'Xt0001371AdA053r02C3\n')
bytearray(b'Xt000138ABd205Cr02C3\n')
bytearray(b'Xt00013A3CdA055r02C3\n')
bytearray(b'Xt00013BCDd2052r02C3\n')
bytearray(b'Xt00013D5Ed2055r02C3\n')
bytearray(b'Xt00013EEFdA058r02C3\n')
bytearray(b'Xt00014080d2056r02C3\n')
bytearray(b'Xt00014211dA059r02C3\n')
bytearray(b'Xt000143A2dA056r02C3\n')
bytearray(b'Xt00014533d2058r02C3\n')
bytearray(b'Xt000146C4dA058r02C4\n')
bytearray(b'Xt00014855d2056r02C4\n')
bytearray(b'Xt000149E6dA055r02C4\n')
bytearray(b'Xt00014B77d2053r02C4\n')
bytearray(b'Xt00014D08dA052r02C4\n')
b

Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
KeyboardInterrupt: 


In [86]:
from machine import UART, Pin
import time, socket, urandom
from BNO055_class import BNO055
from UDPblackviewphone import connectActivePhone

fconfig = dict(x.strip().split(None, 1)  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)

def MakeBNO055(dev):
    d = dev.split(",")
    u = UART(int(d[0]), baudrate=115200, rx=int(d[1][2:]), tx=int(d[2][2:]))
    for i in range(5):
        try:
            print(u, d[3])
            return BNO055(u, d[3])
        except OSError as e:
            print(e)
        print("fail", i, d)
        time.sleep(0.5)
dev1 = MakeBNO055(fconfig["dev1"])


deviceletter = fconfig["deviceletter"]
c = fconfig["connection0"].split(",")
hotspots = { c[0].encode(): (c[1].encode(), c[2], int(c[3])) }
androidipnumber = c[2]
portnumber = int(c[3])
print("deviceletter", deviceletter)

while not connectActivePhone(pinled, hotspots):
    for j in range(3):
        for i in range(7):
            pinled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

import socket, urandom
prevflushstamp = 0
nextledonstamp = 0

ubs = bytearray("Ut00000000i00\n")
mubs = memoryview(ubs)

while True:
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        dwrite = s.write
        while True:
            tstamp = time.ticks_ms()
            if tstamp > nextledonstamp:
                pv = (0 if pinled.value() else 1)
                pinled.value(pv)
                mubs[2:10] = b"%08X" % tstamp
                mubs[12] = 48+pv #'0'or'1'
                dwrite(ubs)
                nextledonstamp = tstamp + urandom.randint(5000, 10000)//(10 if pv else 1)
            if dev1 is not None:
                b1 = dev1.readhexlifyBNO055(10)
                if b1 is not None:
                    dwrite(b1)

            try:
                bs = readi2cdevices(tstamp)
                if bs is not None:
                    dwrite(bs)
            except OSError:
                pass
                
    except OSError as e:
        print("OSError", e)
        time.sleep(2)


UART(1, baudrate=115201, bits=8, parity=None, stop=1, tx=17, rx=16, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=1) Z
BADTEMP
fail 0 ['1', 'rx16', 'tx17', 'Z']
UART(1, baudrate=115201, bits=8, parity=None, stop=1, tx=17, rx=16, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=1) Z
Temp21
deviceletter @
{b'JulianS9': (b'beckaaaa', '192.168.43.1', 9042)}
(b'JulianS9', b'n\xc7\xec\xb9\xc5\x11', 1, -45, 3, False)
.<socket>
OSError [Errno 104] ECONNRESET
<socket>
OSError [Errno 104] ECONNRESET
.<socket>
OSError [Errno 104] ECONNRESET
<socket>
OSError [Errno 104] ECONNRESET
<socket>
OSError [Errno 104] ECONNRESET
<socket>
b'Hello from Android thread 0\n'
None bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
None bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

36 bytearray(b'\xbb"\x01\x00\x02\x00\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x02\x00\x01\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\xfe\xff\x03\x00\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x02\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x00\x00\xff\xff\x00\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\xff\xff\xff\xff\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x01\x00\xff\xff\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x01\x00\xff\xff\xfd\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x00\x00\xfe\xff\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x00\x00\x01\x00\x00\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x01\x00\xff\xff\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x02\x00\x00\x00\x02\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\xfe\xff\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(

36 bytearray(b'\x164\xbb"\x02\x00\xff\xff\xff\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x02\x00\x00\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xff\xff\x01\x00\x00\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\xff\xff\x00\x00\x00\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x00\x00\x02\x00\x00\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\xff\xff\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x01\x00\x00\x00\xff\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x01\x00\x00\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x01\x00\x00\x00\x00\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x01\x00\x00\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x00\x00\x03\x00\xff\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x00\x00\x00\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xff\xff\x02\x00\xff\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\x00\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(

36 bytearray(b'\xbb"\x01\x00\xfe\xff\xfe\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x01\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x00\x00\xfe\xff\xfe\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\xff\xff\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\xff\xff\x00\x00\xff\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x02\x00\x01\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x00\x00\xff\xff\x02\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x02\x00\x02\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x00\x00\xff\xff\xff\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\xff\xff\xc5\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x01\x00\xfe\xff\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x01\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
2 bytearray(b'\xee\x07\x01\x00\xfe\xff\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x01\x00\x00\x00\xc5\x00k\xff\xb4\x03\x164')
36 bytearra

36 bytearray(b'\x164\xbb"\x00\x00\xff\xff\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\x01\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xfd\xff\xff\xff\xfe\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\xff\xff\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x01\x00\xff\xff\xfd\xff~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x03\x00\x01\x00\xfe\xff\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x01\x00\x02\x00\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\x00\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x03\x00\xff\xff\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\x02\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x03\x00\xfe\xff\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x01\x00\x00\x00\x01\x00\xc5\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xfe\xff\x02\x00\x01\x00~\x16\xb9\x00q\xff{?\x18\xfb\x7f\xf9\x11\x00\x00\x00\x00\x00\xff\xff\xc5\x00k\xff\xb4\x03')
36 bytearray(

...

36 bytearray(b'\x164\xbb"\xfd\xff\x02\x00\xfc\xff|\x16\xbd\x00r\xffx?\x1c\xfb`\xf9 \x00\x05\x00\x04\x00\x01\x00\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xfb\xff\xfd\xff\x04\x00|\x16\xbd\x00r\xffx?\x1c\xfb`\xf9 \x00\x02\x00\x01\x00\xfb\xff\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\n\x00\xff\xff\x03\x00|\x16\xbd\x00r\xffx?\x1d\xfb`\xf9 \x00\x00\x00\x01\x00\xfe\xff\xc9\x00l\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xfd\xff\x00\x00\x06\x00|\x16\xbd\x00r\xffx?\x1d\xfb`\xf9 \x00\xfd\xff\xff\xff\x00\x00\xc9\x00l\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x05\x00\xff\xff\xfe\xff|\x16\xbd\x00r\xffx?\x1d\xfb`\xf9 \x00\x01\x00\x00\x00\x03\x00\xc9\x00l\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xfd\xff\xfc\xff\xfc\xff|\x16\xbd\x00r\xffx?\x1d\xfb`\xf9 \x00\x01\x00\x00\x00\x01\x00\xc9\x00l\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xf7\xff\xfd\xff\xfb\xff|\x16\xbd\x00r\xffx?\x1d\xfb`\xf9 \x00\x05\x00\x02\x00\xfe\xff\xc9\x00l\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xfa\xff\x01\x00\xf7\xff|\x16\xb

36 bytearray(b'\x164\xbb"\x00\x00\x03\x00\x03\x00|\x16\xbd\x00q\xffx?\x1b\xfb`\xf9"\x00\x05\x00\x05\x00\xfb\xff\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x06\x00\x05\x00\x07\x00|\x16\xbd\x00q\xffx?\x1b\xfb`\xf9"\x00\x00\x00\x02\x00\x03\x00\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\r\x00\x05\x00\t\x00|\x16\xbd\x00r\xffx?\x1c\xfb`\xf9"\x00\x00\x00\x00\x00\x06\x00\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x07\x00\xfe\xff\x08\x00|\x16\xbd\x00r\xffx?\x1c\xfb`\xf9"\x00\xfe\xff\xfe\xff\x03\x00\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xfd\xff\xfb\xff\xfd\xff|\x16\xbd\x00r\xffx?\x1c\xfb`\xf9"\x00\x00\x00\xff\xff\x01\x00\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xf6\xff\xfe\xff\xf5\xff|\x16\xbd\x00q\xffx?\x1b\xfb`\xf9!\x00\x06\x00\x00\x00\xfb\xff\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\xff\xff\x01\x00\x00\x00|\x16\xbd\x00q\xffx?\x1a\xfb`\xf9 \x00\x03\x00\x02\x00\xfd\xff\xc9\x00k\xff\xb4\x03')
36 bytearray(b'\x164\xbb"\x0c\x00\x06\x00\x06\x00|\x16\xbd\

36 bytearray(b'\xbb"\x08\x00\x07\x00\x08\x00{\x16\xbd\x00q\xffw?\x15\xfba\xf9%\x00\x02\x00\x00\x00\x05\x00\xc8\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x02\x00\x0c\x00\xf9\xff{\x16\xbc\x00q\xffw?\x15\xfbb\xf9%\x00\x01\x00\x04\x00\x05\x00\xc8\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\xfd\xff\x01\x00\xfd\xff{\x16\xbc\x00q\xffw?\x15\xfbb\xf9%\x00\x00\x00\x05\x00\x01\x00\xc8\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\xfc\xff\x03\x00\xfb\xff{\x16\xbc\x00q\xffw?\x14\xfbb\xf9%\x00\x03\x00\x03\x00\x00\x00\xc8\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x05\x00\xfe\xff\x06\x00{\x16\xbc\x00q\xffw?\x14\xfbb\xf9%\x00\x00\x00\x00\x00\x00\x00\xc8\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\x07\x00\x02\x00\x0e\x00{\x16\xbc\x00q\xffw?\x14\xfbb\xf9&\x00\xfa\xff\xfe\xff\x06\x00\xc8\x00j\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\n\x00\xfa\xff\x03\x00{\x16\xbc\x00q\xffw?\x15\xfbb\xf9&\x00\xfe\xff\xfe\xff\x07\x00\xc8\x00k\xff\xb4\x03\x164')
36 bytearray(b'\xbb"\xfa\xff\xfd\xff\xf7\xff{\x16\xbc\x00

36 bytearray(b'\xbb"\xf3\xff\xfb\xff\xfc\xff|\x16\xbe\x00q\xffw?\x1a\xfbW\xf9 \x00\x03\x00\x01\x00\xfd\xff\xca\x00k\xff\xb3\x03\x164')
36 bytearray(b'\xee\x07\xbb"\x02\x00\xfa\xff\x03\x00|\x16\xbe\x00q\xffw?\x1a\xfbW\xf9\x1f\x00\x01\x00\xff\xff\xf8\xff\xca\x00k\xff\xb3\x03')
36 bytearray(b'\x164\xbb"\x06\x00\xfd\xff\x00\x00|\x16\xbe\x00q\xffw?\x1a\xfbW\xf9\x1f\x00\x05\x00\x02\x00\x08\x00\xca\x00k\xff\xb3\x03')
36 bytearray(b'\x164\xbb"\x15\x00\xf9\xff\x01\x00|\x16\xbe\x00r\xffw?\x1c\xfbW\xf9\x1f\x00\x03\x00\x02\x00\x01\x00\xca\x00l\xff\xb3\x03')
36 bytearray(b'\x164\xbb"\xfa\xff\xf8\xff\xfc\xff|\x16\xbe\x00r\xffw?\x1c\xfbW\xf9\x1f\x00\x07\x00\x03\x00\xfd\xff\xca\x00l\xff\xb3\x03')
36 bytearray(b'\x164\xbb"\xf2\xff\x02\x00\xf2\xff|\x16\xbe\x00q\xffw?\x1b\xfbW\xf9\x1e\x00\x0c\x00\x08\x00\x00\x00\xca\x00k\xff\xb3\x03')
36 bytearray(b'\x164\xbb"\t\x00\x01\x00\x06\x00|\x16\xbe\x00q\xffw?\x1b\xfbW\xf9\x1e\x00\x01\x00\x00\x00\xfd\xff\xca\x00k\xff\xb3\x03')
36 bytearray(b'\x164\xbb"\r\x00\xfb\

Traceback (most recent call last):
  File "<stdin>", line 65, in <module>
  File "BNO055_class.py", line 91, in readhexlifyBNO055
KeyboardInterrupt: 
